In [1]:
import json
import keras
from keras.models import load_model
import numpy as np
from __future__ import unicode_literals # allows to use unicode strings
import spacy
import markov
from random import choice, randint

Using TensorFlow backend.


In [2]:
model = load_model('models/metamorphosis.h5')
nlp = spacy.load('en')

In [40]:
maxlen = 40
text = open('source_text/gutenberg.txt').read().lower() # read the file and convert to lowercase 

chars = sorted(list(set(text)))
# what position does each character exist at in the prev list
char_indices = dict((c,i) for i, c in enumerate(chars))
indices_char = dict((i,c) for i,c in enumerate(chars))

In [55]:
model = markov.build_model(text, 1)
" ".join(markov.generate(model, 1, ['continue']))

"continue our guns to see france and still nicer,” said madame de la scala, hum! 'arms: azure, its obscurity concerning which she have ever actually throb always right. one whom he in his own sovereign and commend the libyan floods overflowed with the whalemen call it. harriet also. i saw them of the square, too, got home at every moment there is far off. his amputated above all, and here comes to see them.... but no farther.” “what can give it was forced into the andras[65] the charwoman just wedged in your little there was never of the occasion served. he"

In [11]:
# helper function to sample an index from a probability array
# This comes directly from
# https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py
def sample(preds, temperature=0.6):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [13]:
## sentence = 'alice was beggining to get very tired of'
#sentence = 'sturn left andturn left andturn left and'
initial_route = 'turn right and'
# Grab diversity
diversity = float(0.5)
# how many chars to generate
length = int(100)
# Start with empty generated string
generated = ''
line = ''

start_index = randint(0, len(text)-maxlen-1)

while len(initial_route) < maxlen:
    initial_route = initial_route + " " + text[start_index:start_index+maxlen]
if len(initial_route) > maxlen:
    initial_route = initial_route[:maxlen]

sentence = initial_route

for i in range(length):
    # First vectorize the text we are feeding in
    x = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x[0, t, char_indices[char]] = 1.
    # Then get a probability map of possible predictions of the next char
    preds = model.predict(x, verbose=0)[0]
    # Pick one
    next_index = sample(preds, diversity)
    # What character is it?
    next_char = indices_char[next_index]
    # Add it
    generated += next_char
    # Now feed in the previous sentence minus the first char plus the next char
    sentence = sentence[1:] + next_char
    # nlp for the sentence
    line = ' '.join(initial_route.split()[:3]) + ' ' +' '.join(generated[40:].split())
    
print(line)
print("=====")
doc = nlp(line)
nouns = [item.text for item in doc if item.pos_ == 'NOUN']
adjs = [item.text for item in doc if item.pos_ == 'ADJ']    
line = line.partition(choice(nouns))[0] + choice(nouns) + '.'
line = " ".join(line.split()).capitalize()
print(line)


turn right and e whole for their wort of his breakfasting the from the floo
=====
Turn right and e whole for their wort of his breakfasting the from the floo.
